# Baseline Trading Dashboard — Clean & Compact
one-file notebook version; minimal markdown

## Setup
simple imports

In [ ]:
# minimal imports
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path
from datetime import datetime


## Config
basic params

In [ ]:
# config
CONFIG = dict(initial_capital=100000, transaction_cost=0.001, target_rows=5000)
OUTDIR = Path('notebooks'); OUTDIR.mkdir(parents=True, exist_ok=True)


## Engine
validators, core, metrics, wrapper

In [ ]:
# engine
def _v_sig(x):
    x = np.asarray(x, float)
    x = np.where(np.isnan(x), 0, x)
    return np.clip(np.where(np.abs(x) < 0.05, 0, x), -1, 1)

def _v_ret(r):
    r = np.asarray(r, float)
    r = np.where(np.isnan(r), 0, r)
    return np.clip(r, -0.2, 0.2)

def _core(sig, ret, fee, cash):
    n = len(sig); sig = _v_sig(sig); ret = _v_ret(ret)
    eq = np.zeros(n); pos = np.zeros(n); eq[0] = cash; p = 0.0
    for i in range(1, n):
        tgt = float(sig[i]); d = tgt - p
        if abs(d) > 0.01:
            c = min(abs(d) * fee, 0.05); cash *= (1 - c); p = tgt
        if abs(p) > 0.01:
            dr = np.clip(p * ret[i], -0.1, 0.1); cash *= (1 + dr)
        pos[i] = p; eq[i] = max(cash, 0)
        if cash < 0.01 * eq[0]: pos[i:] = 0; eq[i:] = cash; break
    return eq, pos

def _metrics(eq):
    if len(eq) < 2: return _zeros()
    eq = np.asarray(eq)[np.asarray(eq) > 0]
    if len(eq) < 2: return _zeros()
    tot = np.clip((eq[-1] - eq[0]) / eq[0], -0.95, 10.0)
    dr = np.diff(eq)/eq[:-1]; dr = dr[np.isfinite(dr)]
    if len(dr) == 0: return _zeros()
    mu, sd = dr.mean(), dr.std()
    shp = np.clip((mu/sd)*np.sqrt(252), -5, 5) if sd>0 else 0
    peak = np.maximum.accumulate(eq); dd = (peak-eq)/peak
    mdd = float(min(dd.max(), 0.99))
    yrs = len(eq)/252; ann = np.clip((1+tot)**(1/yrs)-1, -0.8, 3.0) if yrs>0.1 else tot
    calmar = np.clip(ann/mdd if mdd>1e-3 else 0, -10, 10)
    wins, losses = dr[dr>0], dr[dr<0]
    wr = len(wins)/len(dr) if len(dr)>0 else 0.5
    aw = wins.mean() if len(wins)>0 else 0
    al = abs(losses.mean()) if len(losses)>0 else 0.01
    pf = np.clip(aw/al if al>0 else 1, 0.1, 10)
    v95 = np.percentile(dr, 5) if len(dr)>0 else 0
    cvar = dr[dr<=v95].mean() if len(dr)>0 else 0
    return dict(total_return=float(tot), annual_return=float(ann), sharpe_ratio=float(shp),
                calmar_ratio=float(calmar), max_drawdown=float(mdd), win_rate=float(wr),
                profit_factor=float(pf), var_95=float(v95), cvar_95=float(cvar),
                volatility=float(sd*np.sqrt(252)), mean_return=float(mu))

def _zeros():
    return dict(total_return=0.0, annual_return=0.0, sharpe_ratio=0.0, calmar_ratio=0.0,
                max_drawdown=0.0, win_rate=0.5, profit_factor=1.0, var_95=0.0,
                cvar_95=0.0, volatility=0.0, mean_return=0.0)

class Engine:
    def __init__(self, cfg): self.cfg = cfg
    def run(self, strat, df, sym):
        r = df['close'].pct_change().fillna(0).values
        s = strat.generate_signals(df).values
        n = min(len(s), len(r)); s, r = s[:n], r[:n]
        eq, pos = _core(s, r, self.cfg['transaction_cost'], self.cfg['initial_capital'])
        m = _metrics(eq); n_tr = int((np.abs(np.diff(pos))>0.01).sum()); m['n_trades']=n_tr
        return dict(strategy=strat.name, symbol=sym, metrics=m, success=True, equity_curve=eq, positions=pos)


## Strategies
simple baselines

In [ ]:
# strategies (concise)
class BuyAndHold:
    def __init__(self, config=None): self.name='BuyAndHold'
    def generate_signals(self, df): return pd.Series(1.0, index=df.index)

class MovingAverageCrossover:
    def __init__(self, fast=20, slow=50, ma_type='SMA', config=None):
        self.name=f"{ma_type}_{fast}_{slow}"; self.f=fast; self.s=slow; self.t=ma_type
    def generate_signals(self, df):
        c=df['close']; f=c.rolling(self.f, min_periods=1).mean() if self.t=='SMA' else c.ewm(span=self.f).mean()
        s=c.rolling(self.s, min_periods=1).mean() if self.t=='SMA' else c.ewm(span=self.s).mean()
        sig=pd.Series(0.0,index=df.index); sig[f>s]=1.0; sig[f<=s]=-1.0
        return sig.rolling(3, center=True).mean().ffill().fillna(0)

class RSIStrategy:
    def __init__(self, period=14, lo=30, hi=70, config=None):
        self.name=f"RSI_{period}"; self.p=period; self.lo=lo; self.hi=hi
    def generate_signals(self, df):
        d=df['close'].diff(); g=d.where(d>0,0).rolling(self.p).mean(); l=(-d.where(d<0,0)).rolling(self.p).mean()
        rsi=100-(100/(1+g/(l+1e-10))); s=pd.Series(0.0,index=df.index); s[rsi<self.lo]=1.0; s[rsi>self.hi]=-1.0; return s

class BollingerBands:
    def __init__(self, period=20, k=2.0, config=None):
        self.name=f"BB_{period}_{k}"; self.p=period; self.k=k
    def generate_signals(self, df):
        c=df['close']; m=c.rolling(self.p, min_periods=1).mean(); sd=c.rolling(self.p, min_periods=1).std()
        up, lo = m+self.k*sd, m-self.k*sd; s=pd.Series(0.0,index=df.index); s[c<=lo]=1.0; s[c>=up]=-1.0; return s

class MomentumStrategy:
    def __init__(self, lookback=20, config=None): self.name=f"Momentum_{lookback}"; self.lb=lookback
    def generate_signals(self, df):
        mom=df['close'].pct_change(self.lb); s=pd.Series(0.0,index=df.index); s[mom>0.02]=1.0; s[mom<-0.02]=-1.0; return s

class MACDStrategy:
    def __init__(self, fast=12, slow=26, signal=9, config=None):
        self.name=f"MACD_{fast}_{slow}_{signal}"; self.f=fast; self.s=slow; self.sig=signal
    def generate_signals(self, df):
        c=df['close']; ef=c.ewm(span=self.f).mean(); es=c.ewm(span=self.s).mean()
        m=ef-es; sl=m.ewm(span=self.sig).mean(); s=pd.Series(0.0,index=df.index); s[m>sl]=1.0; s[m<=sl]=-1.0; return s


## Data
sample if no file

In [ ]:
# data: load or synthesize
def _sample():
    np.random.seed(42)
    syms={'BTCUSD':(45000,0.03),'ETHUSD':(3000,0.04),'SPY':(450,0.015),'QQQ':(380,0.02)}
    idx=pd.date_range('2023-01-01','2024-01-01',freq='D'); out={}
    for s,(p,vol) in syms.items():
        n=len(idx); r=np.random.normal(0.0005,vol,n); px=[p]
        for x in r[1:]: px.append(px[-1]*(1+x))
        df=pd.DataFrame({'close':px,
                         'open':np.array(px)*(1+np.random.normal(0,0.001,n)),
                         'high':np.array(px)*(1+np.abs(np.random.normal(0,0.005,n))),
                         'low': np.array(px)*(1-np.abs(np.random.normal(0,0.005,n))),
                         'volume':np.random.lognormal(10,0.5,n)}, index=idx)
        out[s]=df.dropna()
    return out

def load_data():
    # simplified: always sample here; plug in your pickle path if available
    return _sample()


## Stats
simple tests

In [ ]:
# stats: t-tests + anova (quiet)
def stats_tests(df):
    res={}
    bh=df[df['strategy_name']=='BuyAndHold']['total_return'].values
    comps={}
    for s in df['strategy_name'].unique():
        if s=='BuyAndHold': continue
        x=df[df['strategy_name']==s]['total_return'].values
        if len(x)>1 and len(bh)>1:
            t,p=stats.ttest_ind(x,bh); comps[s]={'t':float(t),'p':float(p),'mean_diff':float(x.mean()-bh.mean())}
    if comps: res['vs_buyhold']=comps
    groups=[g['total_return'].values for _,g in df.groupby('strategy_name') if len(g)>0]
    if len(groups)>2:
        f,p=stats.f_oneway(*groups); res['anova']={'f':float(f),'p':float(p)}
    if 'asset_type' in df.columns:
        c=df[df['asset_type']=='Crypto']['total_return'].values
        e=df[df['asset_type']=='Equity']['total_return'].values
        if len(c)>1 and len(e)>1:
            t,p=stats.ttest_ind(c,e); res['market']={'crypto_mean':float(c.mean()),'equity_mean':float(e.mean()),'t':float(t),'p':float(p)}
    return res


## Dashboard
8 panels; matplotlib-only

In [ ]:
# dashboard (matplotlib only)
def dashboard(df):
    if df.empty: return None
    fig = plt.figure(figsize=(20,16)); gs = fig.add_gridspec(4,3, hspace=0.35, wspace=0.25)

    # 1 heatmap
    ax1 = fig.add_subplot(gs[0,:])
    try:
        piv = df.pivot_table(values='total_return', index='strategy_name', columns='symbol', aggfunc='mean')
        if piv.size>0:
            im=ax1.imshow(piv.values, aspect='auto'); fig.colorbar(im, ax=ax1, label='Total Return')
            ax1.set_xticks(range(piv.shape[1])); ax1.set_xticklabels(piv.columns, rotation=0)
            ax1.set_yticks(range(piv.shape[0])); ax1.set_yticklabels([s.replace('_',' ') for s in piv.index])
            ax1.set_title('Strategy Performance Heatmap'); ax1.set_xlabel('Asset'); ax1.set_ylabel('Strategy')
        else: ax1.text(0.5,0.5,'Insufficient data', ha='center', va='center')
    except Exception as e:
        ax1.text(0.5,0.5,f'Heatmap error: {e}', ha='center', va='center')

    # 2 Sharpe
    ax2 = fig.add_subplot(gs[1,0])
    try:
        s = df.groupby('strategy_name')['sharpe_ratio'].mean().sort_values()
        ax2.barh(range(len(s)), s.values); ax2.set_yticks(range(len(s))); ax2.set_yticklabels([k.replace('_',' ') for k in s.index])
        ax2.set_xlabel('Avg Sharpe'); ax2.set_title('Risk-Adjusted Performance'); ax2.axvline(0, lw=0.5)
    except Exception as e: ax2.text(0.5,0.5,f'Error: {e}', ha='center', va='center')

    # 3 cross-market
    ax3 = fig.add_subplot(gs[1,1])
    try:
        if 'asset_type' in df.columns:
            mp = df.groupby(['asset_type','strategy_name'])['total_return'].mean().unstack()
            if mp.size>0: mp.T.plot(kind='bar', ax=ax3); ax3.set_title('Cross-Market Performance'); ax3.set_xlabel('Strategy'); ax3.set_ylabel('Avg Return'); ax3.axhline(0, lw=0.5)
            else: ax3.text(0.5,0.5,'No cross-market data', ha='center', va='center')
        else: ax3.text(0.5,0.5,'No asset type data', ha='center', va='center')
    except Exception as e: ax3.text(0.5,0.5,f'Error: {e}', ha='center', va='center')

    # 4 risk-return
    ax4 = fig.add_subplot(gs[1,2])
    try:
        for sname in df['strategy_name'].unique():
            d = df[df['strategy_name']==sname]
            if len(d)>0: ax4.scatter(d['max_drawdown'], d['total_return'], label=sname.replace('_',' '))
        ax4.set_xlabel('Max Drawdown'); ax4.set_ylabel('Total Return'); ax4.set_title('Risk-Return'); ax4.axhline(0,lw=0.5); ax4.axvline(0,lw=0.5); ax4.legend(fontsize=8, bbox_to_anchor=(1.05,1), loc='upper left')
    except Exception as e: ax4.text(0.5,0.5,f'Error: {e}', ha='center', va='center')

    # 5 trades
    ax5 = fig.add_subplot(gs[2,0])
    try:
        t = df.groupby('strategy_name')['n_trades'].mean().sort_values()
        ax5.barh(range(len(t)), t.values); ax5.set_yticks(range(len(t))); ax5.set_yticklabels([k.replace('_',' ') for k in t.index])
        ax5.set_xlabel('Avg Trades'); ax5.set_title('Trading Frequency')
    except Exception as e: ax5.text(0.5,0.5,f'Error: {e}', ha='center', va='center')

    # 6 win rate
    ax6 = fig.add_subplot(gs[2,1])
    try:
        w = df.groupby('strategy_name')['win_rate'].mean().sort_values(ascending=False)
        ax6.bar(range(len(w)), w.values); ax6.set_xticks(range(len(w))); ax6.set_xticklabels([k.replace('_',' ') for k in w.index], rotation=45, ha='right', fontsize=8)
        ax6.set_ylabel('Win Rate'); ax6.set_title('Win Rates'); ax6.set_ylim(0,1); ax6.axhline(0.5, lw=0.8)
    except Exception as e: ax6.text(0.5,0.5,f'Error: {e}', ha='center', va='center')

    # 7 distribution
    ax7 = fig.add_subplot(gs[2,2])
    try:
        data=[]; labels=[]
        for sname in df['strategy_name'].unique():
            vals = df[df['strategy_name']==sname]['total_return'].values
            if len(vals)>0: data.append(vals); labels.append(sname.replace('_',' '))
        if data: ax7.boxplot(data, labels=labels); ax7.set_ylabel('Total Return'); ax7.set_title('Return Distribution'); ax7.axhline(0,lw=0.5); plt.setp(ax7.get_xticklabels(), rotation=45, ha='right', fontsize=8)
        else: ax7.text(0.5,0.5,'No distribution data', ha='center', va='center')
    except Exception as e: ax7.text(0.5,0.5,f'Error: {e}', ha='center', va='center')

    # 8 summary table (text)
    ax8 = fig.add_subplot(gs[3,:]); ax8.axis('off')
    try:
        sm = df.groupby('strategy_name').agg(total_return=('total_return',['mean','std']),
                                             sharpe_ratio=('sharpe_ratio','mean'),
                                             max_drawdown=('max_drawdown','mean'),
                                             win_rate=('win_rate','mean'),
                                             n_trades=('n_trades','mean'))
        sm.columns = ['Avg Return','Return Std','Avg Sharpe','Avg MaxDD','Avg Win Rate','Avg Trades']
        txt = 'COMPREHENSIVE STRATEGY PERFORMANCE SUMMARY\n' + '='*80 + '\n\n'
        txt += f"{'Strategy':<20} {'Avg Return':<12} {'Std Dev':<10} {'Sharpe':<8} {'Max DD':<8} {'Win Rate':<10} {'Trades':<8}\n"
        txt += '-'*80 + '\n'
        for sname,row in sm.round(3).iterrows():
            txt += f"{sname:<20} {row['Avg Return']:>10.1%} {row['Return Std']:>8.1%} {row['Avg Sharpe']:>6.2f} {row['Avg MaxDD']:>6.1%} {row['Avg Win Rate']:>8.1%} {row['Avg Trades']:>6.0f}\n"
        ax8.text(0.02,0.95, txt, transform=ax8.transAxes, fontsize=10, family='monospace', va='top')
    except Exception as e: ax8.text(0.5,0.5,f'Summary error: {e}', ha='center', va='center')

    fig.suptitle('Comprehensive Baseline Strategy Analysis Dashboard', fontsize=18, y=0.98)
    fig.tight_layout()
    out = OUTDIR/'04_complete_strategy_dashboard.png'; fig.savefig(out, dpi=150, bbox_inches='tight')
    return out


## Run
execute pipeline; saves results

In [ ]:
# run
def run_all():
    data = load_data()
    eng = Engine(CONFIG)
    strats = [BuyAndHold(),
              MovingAverageCrossover(20,50,'SMA'),
              MovingAverageCrossover(12,26,'EMA'),
              MovingAverageCrossover(5,20,'SMA'),
              RSIStrategy(14,30,70),
              RSIStrategy(21,25,75),
              BollingerBands(20,2.0),
              BollingerBands(10,1.5),
              MomentumStrategy(20),
              MomentumStrategy(10),
              MACDStrategy(12,26,9)]
    rows=[]; all_results=[]
    for sym, df in data.items():
        a_type = 'Crypto' if any(c in sym.upper() for c in ['BTC','ETH','SOL','ADA','XRP']) else 'Equity'
        for s in strats:
            r = eng.run(s, df, sym); r['asset_type']=a_type; r['strategy_name']=s.name; all_results.append(r)
            rows.append(dict(strategy_name=s.name, symbol=sym, asset_type=a_type, **r['metrics']))
    res = pd.DataFrame(rows)
    stats_res = stats_tests(res)
    dash_path = dashboard(res)
    out = dict(results_df=res, all_results=all_results, stats=stats_res, config=CONFIG, timestamp=datetime.now())
    (OUTDIR/'04_complete_fixed_results.pkl').write_bytes(pd.to_pickle(out))
    res.to_csv(OUTDIR/'04_complete_fixed_results.csv', index=False)
    return res, all_results, stats_res, dash_path

# optional: uncomment to run immediately
# results_df, all_results, statistical_results, dash = run_all()
